## 1. Imports


In [36]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

In [37]:
data_path = '../data/results/test_2_balanced/'
folders= ['dt','gnb','lgr','gbt']

## 2. CSVs

In [38]:
def impact_csvs(data_path= 'data/results/',b_or_w = 'Black', folders= ['dt','gnb','lgr','gbt']):

    col_names_eg = []
    col_names_gs = []

    for i,f in enumerate(folders):
        if b_or_w == 'Black':
            path = f'{data_path}{f}/{f}_black_results.csv'
        else:
            path = f'{data_path}{f}/{f}_white_results.csv'

        df = pd.read_csv(path,index_col=0)
        df = df.reset_index()

        col_names_eg.append(f'EG+{f.upper()}')
        col_names_gs.append(f'GS+{f.upper()}')

        if i == 0:
            joined_df = df.iloc[:,-1]
        else:
            joined_df = pd.concat([joined_df, df.iloc[:,-1]], axis=1)

    joined_df.set_axis(folders, axis=1)


    # split dataframe after the two reduction algorithms
    df_eg = joined_df.iloc[:6,:]
    df_gs = pd.concat([joined_df.iloc[0:1,:],joined_df.iloc[6:,:]])

    # set new index
    df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_eg.set_index('Constraint',inplace=True)
    df_gs['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_gs.set_index('Constraint',inplace=True)
    df_eg.columns = col_names_eg
    df_gs.columns = col_names_gs

    df_final = pd.concat([df_eg, df_gs], axis=1)
    print('Group: ',b_or_w,'\n DataFrame: \n',df_final)
    print('A')
    df_final.to_csv(f'{data_path}/{b_or_w}_DI.csv')
    print('B')


In [39]:
impact_csvs(data_path,'Black', folders= ['dt','gnb','lgr','gbt'])
impact_csvs(data_path,'White', folders= ['dt','gnb','lgr','gbt'])

Group:  Black 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated   7.74    9.46    8.31    7.64   7.74    9.46    8.31    7.64
DP          -18.12  -19.95  -18.29  -17.25   3.19    6.96    1.07    3.19
EO            2.12    0.18    0.90    2.70  -3.23  -12.58    8.31   -3.23
EOO           1.54    1.10    1.03    3.19   1.54    1.61   -0.24    1.61
FPER          2.75    1.50    2.63    2.93   1.54    6.96    6.23    1.61
ERP           7.74    9.42    8.33    7.74   7.74    9.43    8.31    7.74
A
B
Group:  White 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated  39.47   37.22   39.04   39.35  39.47   37.22   39.04   39.35
DP           35.82   36.56   36.01   35.49  39.89   39.72   39.96   39.88
EO           38.86   39.27   39.18   38.85  38.89 

C:\Users\HaRi\AppData\Local\Temp\ipykernel_17264\620886726.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\AppData\Local\Temp\ipykernel_17264\620886726.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


## 3. FP/TP/TN/FN Ratios

In [40]:
# Ratios
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts(normalize=True)
    df = df.reset_index()
    df = df.rename(columns= {0:'Ratio'})
    
    df = df.pivot(index='Category', columns='ID')['Ratio']
    df = df.fillna(0)
    df = df.round(decimals = 3)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_ratios.csv')

Classifier:  dt 
 DataFrame: 
 ID        egdpB  egdpW  egeoB  egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                 
FN        0.026  0.188  0.053  0.113   0.090   0.064    0.055    0.090   
FP        0.271  0.020  0.123  0.037   0.067   0.068    0.118    0.041   
TN        0.392  0.216  0.541  0.199   0.597   0.167    0.546    0.195   
TP        0.310  0.577  0.283  0.651   0.246   0.701    0.281    0.674   

ID        egtprpB  egtprpW  gsdpB  gsdpW  gseoB  gseoW  gserpB  gserpW  \
Category                                                                 
FN          0.052    0.110  0.057  0.083  0.043  0.121   0.090   0.045   
FP          0.128    0.036  0.114  0.045  0.164  0.032   0.067   0.067   
TN          0.536    0.200  0.550  0.191  0.500  0.203   0.597   0.169   
TP          0.285    0.655  0.279  0.682  0.294  0.643   0.246   0.719   

ID        gsfprpB  gsfprpW  gstprpB  gstprpW  testB  testW  unmitB  unmitW  
Ca

In [41]:
# Absolute numbers
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts()
    df = df.reset_index()
    df = df.rename(columns= {0:'Number'})
    
    df = df.pivot(index='Category', columns='ID')['Number']
    df = df.fillna(0)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

Classifier:  dt 
 DataFrame: 
 ID         egdpB   egdpW   egeoB   egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                     
FN         343.0  2440.0   692.0  1472.0  1168.0   827.0    720.0   1167.0   
FP        3527.0   255.0  1598.0   476.0   873.0   889.0   1530.0    534.0   
TN        5101.0  2810.0  7030.0  2589.0  7755.0  2176.0   7098.0   2531.0   
TP        4029.0  7495.0  3680.0  8463.0  3204.0  9108.0   3652.0   8768.0   

ID        egtprpB  egtprpW   gsdpB   gsdpW   gseoB   gseoW  gserpB  gserpW  \
Category                                                                     
FN          672.0   1424.0   741.0  1073.0   554.0  1575.0  1168.0   585.0   
FP         1659.0    464.0  1481.0   586.0  2131.0   422.0   873.0   866.0   
TN         6969.0   2601.0  7147.0  2479.0  6497.0  2643.0  7755.0  2199.0   
TP         3700.0   8511.0  3631.0  8862.0  3818.0  8360.0  3204.0  9350.0   

ID        gsfprpB  gsfprpW  gst

# Analyzing Scores

#### Extractig Scores from csv into dataframes

In [42]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}
dfs_eg = {}
dfs_gs = {}
for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    df_eg = pd.concat([df.iloc[:,:4],df.filter(like='eg')],axis=1)
    df_gs = pd.concat([df.iloc[:,:4],df.filter(like='gs')],axis=1)
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white
    dfs_eg[f] = df_eg
    dfs_gs[f] = df_gs
print(dfs_b['dt'])

       testB  unmitB  egdpB  egeoB  egtprpB  egfprpB  egerpB  gsdpB  gseoB  \
0        763     838    838    838      838      838     838    838    838   
1        710     785    785    785      785      785     785    785    785   
2        390     390    390    390      390      390     390    390    390   
3        662     737    737    737      737      737     737    737    737   
4        475     475    475    475      475      475     475    475    475   
...      ...     ...    ...    ...      ...      ...     ...    ...    ...   
12995    452     452    302    452      452      452     452    452    452   
12996    475     475    475    475      475      475     475    475    475   
12997    483     483    483    483      483      483     483    483    483   
12998    511     511    361    511      511      511     511    511    511   
12999    511     511    361    511      511      511     511    511    511   

       gstprpB  gsfprpB  gserpB  
0          838      838     8

### Checking if normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [43]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm")
        if p > 0.01:
            print(col,',p:',p)
    print('Check for norm Distributions done')

Classifier: dt
Check for norm Distributions done
Classifier: gnb
Check for norm Distributions done
Classifier: lgr
Check for norm Distributions done
Classifier: gbt
Check for norm Distributions done


### Mann Whitney U test:

“a two-sample rank test for the difference between two population medians . . . It assumes that the data are independent random samples from two populations that have the same shape.”

In [45]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

#### Variance of Distributions EG vs. GS
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [46]:
p_vals = pd.DataFrame(data={'Constraints': []})
p_signi = pd.DataFrame(data={'Constraints': []})
    
for (c1,df_eg),(c2,df_gs) in zip(dfs_eg.items(),dfs_gs.items()):
    col_signi = []
    col_vals = []
    idx = []
    
    for col_eg,col_gs in zip(df_eg,df_gs):
        
        idx.append(col_eg[2:])
        
        data_eg=df_eg[col_eg].dropna(axis=0)
        data_gs=df_gs[col_gs].dropna(axis=0)
        
        _,p = stats.mannwhitneyu(data_eg, data_gs)
        
        col_vals.append(p)
        if p< 0.05:
            col_signi.append('s')
        else:
            col_signi.append(' ')
            
    p_signi[c1] = col_signi
    p_vals[c1] = col_vals
    
p_vals['Constraints'] = idx
p_vals = p_vals.set_index('Constraints')
p_signi['Constraints'] = idx
p_signi = p_signi.set_index('Constraints')

p_vals = p_vals.round(decimals=3)
print(p_signi)
    
p_vals.to_csv(f'{mwu_path}p_eg_gs.csv')
p_signi.to_csv(f'{mwu_path}significanz_eg_gs.csv')


            dt gnb lgr gbt
Constraints               
stB                       
stW                       
mitB                      
mitW                      
dpB          s   s   s   s
dpW                       
eoB          s   s   s   s
eoW                       
tprpB                     
tprpW                     
fprpB            s   s    
fprpW                     
erpB                      
erpW                      


#### Variance of Distributions Each Model against each other
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [47]:
def p_values_mwu_1model(dfs,model=''):
    p_vals = pd.DataFrame(data={'Classifier': []})
    p_signi = pd.DataFrame(data={'Classifier': []})
    
    for c1,df1 in dfs.items():
        col_signi = []
        col_vals = []
        idx = []
        
        data1 = df1[model].dropna(axis=0)
        
        for c2,df2 in dfs.items():
            idx.append(c2)
            
            data2 =df2[model].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data1, data2)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c1] = col_signi
        p_vals[c1] = col_vals
        
    p_vals['Classifier'] = idx
    p_vals = p_vals.set_index('Classifier')
    p_signi['Classifier'] = idx
    p_signi = p_signi.set_index('Classifier')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_{model}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_{model}.csv')

In [50]:
for col in classifier_dfs['dt']:
    print('\nC:',col)
    p_values_mwu_1model(classifier_dfs,col)


C: testB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: testW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: unmitB
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s        
gbt             s        

C: unmitW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: egdpB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                     s
lgr                      
gbt             s        

C: egdpW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C

#### Variance of Distributions unmitigated v Mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [51]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0)
        
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
        
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [53]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')

Black:
            dtB gnbB lgrB gbtB
Constraints                   
test                          
unmit                         
egdp          s    s    s    s
egeo          s    s    s    s
egtprp        s    s    s    s
egfprp        s    s    s    s
egerp                         
gsdp          s    s    s    s
gseo          s    s         s
gstprp        s    s    s    s
gsfprp        s    s    s    s
gserp                         

White:
            dtW gnbW lgrW gbtW
Constraints                   
test          s    s    s    s
unmit                         
egdp                          
egeo                          
egtprp                        
egfprp                        
egerp                         
gsdp                          
gseo                          
gstprp                        
gsfprp                        
gserp              s          


In [ ]:
stats.pearsonr(data_unmiti, data_miti)